In [2]:
import kaggle 
!kaggle datasets download -d taeefnajib/used-car-price-prediction-dataset

Dataset URL: https://www.kaggle.com/datasets/taeefnajib/used-car-price-prediction-dataset
License(s): Attribution 4.0 International (CC BY 4.0)




  0%|          | 0.00/109k [00:00<?, ?B/s]
100%|██████████| 109k/109k [00:00<00:00, 160kB/s]
100%|██████████| 109k/109k [00:00<00:00, 160kB/s]


In [3]:
import zipfile
zip = zipfile.ZipFile('used-car-price-prediction-dataset.zip')
zip.extractall()
zip.close()

# Import Lib

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [2]:
df = pd.read_csv('used_cars.csv')

**Brand & Model:** Identify the brand or company name along with the specific model of each vehicle.  

**Model Year:** Discover the manufacturing year of the vehicles, crucial for assessing depreciation and technology advancements.  

**Mileage:** Obtain the mileage of each vehicle, a key indicator of wear and tear and potential maintenance requirements.  

**Fuel Type:** Learn about the type of fuel the vehicles run on, whether it's gasoline, diesel, electric, or hybrid.  

**Engine Type:** Understand the engine specifications, shedding light on performance and efficiency.  

**Transmission:** Determine the transmission type, whether automatic, manual, or another variant.  

**Exterior & Interior Colors:** Explore the aesthetic aspects of the vehicles, including exterior and interior color options.  

**Accident History:** Discover whether a vehicle has a prior history of accidents or damage, crucial for informed decision-making.  

**Clean Title:** Evaluate the availability of a clean title, which can impact the vehicle's resale value and legal status.  

**Price:** Access the listed prices for each vehicle, aiding in price comparison and budgeting.  

In [3]:
df.head()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,Ford,Utility Police Interceptor Base,2013,"51,000 mi.",E85 Flex Fuel,300.0HP 3.7L V6 Cylinder Engine Flex Fuel Capa...,6-Speed A/T,Black,Black,At least 1 accident or damage reported,Yes,"$10,300"
1,Hyundai,Palisade SEL,2021,"34,742 mi.",Gasoline,3.8L V6 24V GDI DOHC,8-Speed Automatic,Moonlight Cloud,Gray,At least 1 accident or damage reported,Yes,"$38,005"
2,Lexus,RX 350 RX 350,2022,"22,372 mi.",Gasoline,3.5 Liter DOHC,Automatic,Blue,Black,None reported,NaN,"$54,598"
3,INFINITI,Q50 Hybrid Sport,2015,"88,900 mi.",Hybrid,354.0HP 3.5L V6 Cylinder Engine Gas/Electric H...,7-Speed A/T,Black,Black,None reported,Yes,"$15,500"
4,Audi,Q3 45 S line Premium Plus,2021,"9,835 mi.",Gasoline,2.0L I4 16V GDI DOHC Turbo,8-Speed Automatic,Glacier White Metallic,Black,None reported,NaN,"$34,999"


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4009 entries, 0 to 4008
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   brand         4009 non-null   object
 1   model         4009 non-null   object
 2   model_year    4009 non-null   int64 
 3   milage        4009 non-null   object
 4   fuel_type     3839 non-null   object
 5   engine        4009 non-null   object
 6   transmission  4009 non-null   object
 7   ext_col       4009 non-null   object
 8   int_col       4009 non-null   object
 9   accident      3896 non-null   object
 10  clean_title   3413 non-null   object
 11  price         4009 non-null   object
dtypes: int64(1), object(11)
memory usage: 376.0+ KB


# Data Cleaning

In [5]:
df['clean_title'].unique()

array(['Yes', nan], dtype=object)

In [6]:
def count_dash(df):
    return (df == '–').sum()

dash_counts =count_dash(df)

cols_with_dash = dash_counts[dash_counts > 0].index
df[cols_with_dash] = df[cols_with_dash].replace('–', np.nan)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4009 entries, 0 to 4008
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   brand         4009 non-null   object
 1   model         4009 non-null   object
 2   model_year    4009 non-null   int64 
 3   milage        4009 non-null   object
 4   fuel_type     3794 non-null   object
 5   engine        3964 non-null   object
 6   transmission  4005 non-null   object
 7   ext_col       3994 non-null   object
 8   int_col       3876 non-null   object
 9   accident      3896 non-null   object
 10  clean_title   3413 non-null   object
 11  price         4009 non-null   object
dtypes: int64(1), object(11)
memory usage: 376.0+ KB


In [8]:
df.sample(5)

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
3909,Buick,Cascada Base,2016,"103,814 mi.",Gasoline,200.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,6-Speed A/T,White,Black,At least 1 accident or damage reported,Yes,"$14,995"
916,Porsche,911 Carrera 4S,2018,"31,678 mi.",Gasoline,3.0 Liter,NaN,Silver,NaN,None reported,NaN,"$106,798"
1979,Mercedes-Benz,C-Class C 63 AMG,2012,"103,000 mi.",Gasoline,451.0HP 6.3L 8 Cylinder Engine Gasoline Fuel,7-Speed A/T,Silver,Black,At least 1 accident or damage reported,Yes,"$40,000"
329,Mercedes-Benz,AMG G 63 Base,2019,"19,593 mi.",Gasoline,4.0 Liter Twin Turbo,Automatic,Black,Black,None reported,NaN,"$159,998"
1008,Volvo,C40 Recharge Pure Electric Twin Ultimate,2023,"17,000 mi.",NaN,402.0HP Electric Motor Electric Fuel System,A/T,Gray,Blue,None reported,Yes,"$54,900"


## Mising Values

In [9]:
df.isnull().sum()

brand             0
model             0
model_year        0
milage            0
fuel_type       215
engine           45
transmission      4
ext_col          15
int_col         133
accident        113
clean_title     596
price             0
dtype: int64

In [10]:
df['fuel_type'].unique()

array(['E85 Flex Fuel', 'Gasoline', 'Hybrid', nan, 'Diesel',
       'Plug-In Hybrid', 'not supported'], dtype=object)

In [11]:
df.groupby('fuel_type')['fuel_type'].count()

fuel_type
Diesel             116
E85 Flex Fuel      139
Gasoline          3309
Hybrid             194
Plug-In Hybrid      34
not supported        2
Name: fuel_type, dtype: int64

In [12]:
df['fuel_type'] = df['fuel_type'].fillna(df['fuel_type'].mode()[0])

In [13]:
df['brand'].nunique()

57

In [ ]:
df.groupby('brand')['price'].mean().sort_values(ascending=False).head(10)

In [19]:
df['accident'].unique()

array([1, 0], dtype=int64)

In [18]:
df.groupby('accident')['accident'].count()

accident
0    3023
1     986
Name: accident, dtype: int64

In [16]:
df['accident'] = df['accident'].fillna(df['accident'].mode()[0])

In [17]:
accident_map = {'At least 1 accident or damage reported': 1, 'None reported': 0}
df['accident'] =df['accident'].apply(lambda x : accident_map[x])

In [20]:
df.groupby('clean_title')['clean_title'].count()

clean_title
Yes    3413
Name: clean_title, dtype: int64

In [21]:
df['clean_title'] = df['clean_title'].fillna('No')

In [22]:
df['brand'].unique()

array(['Ford', 'Hyundai', 'Lexus', 'INFINITI', 'Audi', 'Acura', 'BMW',
       'Tesla', 'Land', 'Aston', 'Toyota', 'Lincoln', 'Jaguar',
       'Mercedes-Benz', 'Dodge', 'Nissan', 'Genesis', 'Chevrolet', 'Kia',
       'Jeep', 'Bentley', 'Honda', 'Lucid', 'MINI', 'Porsche', 'Hummer',
       'Chrysler', 'Volvo', 'Cadillac', 'Lamborghini', 'Maserati',
       'Volkswagen', 'Subaru', 'Rivian', 'GMC', 'RAM', 'Alfa', 'Ferrari',
       'Scion', 'Mitsubishi', 'Mazda', 'Saturn', 'Bugatti', 'Polestar',
       'Rolls-Royce', 'McLaren', 'Buick', 'Lotus', 'Pontiac', 'FIAT',
       'Karma', 'Saab', 'Mercury', 'Plymouth', 'smart', 'Maybach',
       'Suzuki'], dtype=object)

In [23]:
df['transmission'].fillna(df['transmission'].mode()[0], inplace=True)
df['engine'].fillna(df['engine'].mode()[0], inplace=True)
df['ext_col'].fillna(df['ext_col'].mode()[0], inplace=True)
df['int_col'].fillna(df['int_col'].mode()[0], inplace=True)

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4009 entries, 0 to 4008
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   brand         4009 non-null   object
 1   model         4009 non-null   object
 2   model_year    4009 non-null   int64 
 3   milage        4009 non-null   object
 4   fuel_type     4009 non-null   object
 5   engine        4009 non-null   object
 6   transmission  4009 non-null   object
 7   ext_col       4009 non-null   object
 8   int_col       4009 non-null   object
 9   accident      4009 non-null   int64 
 10  clean_title   4009 non-null   object
 11  price         4009 non-null   object
dtypes: int64(2), object(10)
memory usage: 376.0+ KB


# Feature Engineering

In [25]:
df.sample(5)

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
1852,BMW,M3 Base,2011,"43,000 mi.",Gasoline,414.0HP 4.0L 8 Cylinder Engine Gasoline Fuel,6-Speed M/T,Black,Black,0,Yes,"$50,900"
3506,Toyota,RAV4 Base,2009,"78,500 mi.",Gasoline,179.0HP 2.5L 4 Cylinder Engine Gasoline Fuel,4-Speed A/T,Silver,Black,0,Yes,"$12,000"
793,Alfa,Romeo Giulia Ti,2019,"21,400 mi.",Gasoline,280.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,8-Speed A/T,White,Black,0,Yes,"$28,500"
29,Audi,Q5 2.0T Premium Plus,2017,"81,880 mi.",E85 Flex Fuel,2.0L I4 16V GDI DOHC Turbo Flexible Fuel,8-Speed Automatic,Mythos Black Metallic,Black,0,Yes,"$21,375"
2283,Ford,Mustang GT Premium,2019,"31,600 mi.",Gasoline,460.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,6-Speed M/T,Blue,Gray,0,Yes,"$41,000"


In [26]:
df['transmission_type'] = df['transmission'].apply(lambda x: 'Automatic' if 'a/t' in x.lower() or 'automatic' in x.lower() else 'Manual')

In [27]:
df['milage'] = df['milage'].str.replace(r'[^\d.]', '', regex=True).astype(float)
df['price'] = df['price'].str.replace(r'[^\d.]', '', regex=True).astype(float)

In [28]:
def extract_litre_eng(engine_desc):
    match = re.search(r'(\d+(\.\d+)?)\s*[Ll](?:iter)?', engine_desc)
    return float(match.group(1)) if match else None

df['engine_litre'] = df['engine'].apply(extract_litre_eng)
df['engine_litre'].fillna(df['engine_litre'].mode()[0], inplace=True)

In [29]:
def extract_hp(engine_desc):
    match  = re.search(r'(\d+(\.\d+)?)HP', engine_desc)
    return float(match.group(1)) if match else None

df_hp = pd.DataFrame()
df_hp['engine'] = df['engine']
df_hp['hp'] = df_hp['engine'].apply(extract_hp)
print('Hp Mean : ',df_hp['hp'].mean())
print('Hp Mode : ',df_hp['hp'].mode()[0])

Hp Mean :  332.20649796938454
Hp Mode :  300.0


In [30]:
df['hp'] = df['engine'].apply(extract_hp)
df['hp'].fillna(df['hp'].mode()[0], inplace=True)

In [31]:
def extract_cylinders(engine_desc):
    match = re.search(r'\b([VvIi]\d+)\b', engine_desc)
    if match:
        config = match.group(1)
        return int(config[1:])  

    match = re.search(r'(\d+(\.\d+)?)\s*L', engine_desc)
    if match:
        displacement = float(match.group(1))
        if displacement <= 2.5:
            return 4  
        elif displacement <= 3.8:
            return 6  
        else:
            return 8  
    return None


df['cylinders'] = df['engine'].apply(extract_cylinders)
df['cylinders'].fillna(df['cylinders'].mode()[0], inplace=True)

In [32]:
df['transmission'].unique()

array(['6-Speed A/T', '8-Speed Automatic', 'Automatic', '7-Speed A/T',
       'F', 'A/T', '8-Speed A/T', 'Transmission w/Dual Shift Mode',
       '9-Speed Automatic', '6-Speed M/T', '10-Speed A/T', '9-Speed A/T',
       '5-Speed A/T', '1-Speed A/T', 'Automatic CVT',
       '7-Speed Automatic with Auto-Shift', 'CVT-F', 'M/T',
       '6-Speed Automatic with Auto-Shift', '10-Speed Automatic',
       'CVT Transmission', '4-Speed A/T', '6-Speed Automatic',
       '4-Speed Automatic', '7-Speed M/T',
       '8-Speed Automatic with Auto-Shift', '5-Speed Automatic',
       '8-SPEED AT', '1-Speed Automatic', '5-Speed M/T', 'Manual',
       '6-Speed Manual', 'Variable', '2', '7-Speed Manual',
       'Automatic, 9-Spd 9G-Tronic', 'Automatic, 8-Spd',
       'Automatic, 8-Spd Sport w/Sport & Manual Modes',
       'Auto, 6-Spd w/CmdShft', 'Transmission Overdrive Switch',
       '7-Speed Automatic', 'Automatic, 8-Spd PDK Dual-Clutch',
       'Automatic, 8-Spd M STEPTRONIC w/Drivelogic, Sport & Manual 

In [33]:
def extract_speed(transmission_desc):
    match = re.search(r'(\d+)[- ]?(Speed|Spd)', transmission_desc, re.IGNORECASE)
    if match:
        return match.group(1)
    else:
        return None

extract_speed('6-Speed A/T')

'6'

In [34]:
df['transmission_speed'] = df['transmission'].apply(extract_speed)

In [35]:
df[df['transmission_speed'].isnull()]['transmission'].unique()

array(['Automatic', 'F', 'A/T', 'Transmission w/Dual Shift Mode',
       'Automatic CVT', 'CVT-F', 'M/T', 'CVT Transmission', 'Manual',
       'Variable', '2', 'Transmission Overdrive Switch',
       'Single-Speed Fixed Gear', 'SCHEDULED FOR OR IN PRODUCTION'],
      dtype=object)

In [36]:
df.loc[df['transmission'] == '2', 'transmission_speed'] = '2'
df.loc[df['transmission'] == 'Single-Speed Fixed Gear', 'transmission_speed'] = '1'

In [37]:
df['transmission_speed'].fillna('5', inplace= True)

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4009 entries, 0 to 4008
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   brand               4009 non-null   object 
 1   model               4009 non-null   object 
 2   model_year          4009 non-null   int64  
 3   milage              4009 non-null   float64
 4   fuel_type           4009 non-null   object 
 5   engine              4009 non-null   object 
 6   transmission        4009 non-null   object 
 7   ext_col             4009 non-null   object 
 8   int_col             4009 non-null   object 
 9   accident            4009 non-null   int64  
 10  clean_title         4009 non-null   object 
 11  price               4009 non-null   float64
 12  transmission_type   4009 non-null   object 
 13  engine_litre        4009 non-null   float64
 14  hp                  4009 non-null   float64
 15  cylinders           4009 non-null   float64
 16  transm

In [39]:
df.sample(5)

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,transmission_type,engine_litre,hp,cylinders,transmission_speed
3323,Ford,F-150 Lariat,2016,25000.0,E85 Flex Fuel,385.0HP 5.0L 8 Cylinder Engine Flex Fuel Capab...,6-Speed A/T,Gray,Black,0,Yes,65000.0,Automatic,5.0,385.0,8.0,6
331,Hyundai,Veloster Base,2013,800.0,Gasoline,132.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Gray,0,Yes,9800.0,Manual,1.6,132.0,4.0,5
3503,GMC,Canyon Elevation Standard,2022,16500.0,Gasoline,308.0HP 3.6L V6 Cylinder Engine Gasoline Fuel,8-Speed A/T,Gray,Gray,0,Yes,32900.0,Automatic,3.6,308.0,6.0,8
2995,BMW,Z4 2.5i,2004,94500.0,Gasoline,184.0HP 2.5L Straight 6 Cylinder Engine Gasoli...,5-Speed M/T,Gray,Black,0,Yes,9000.0,Manual,2.5,184.0,4.0,5
741,Dodge,Caliber Express,2010,102681.0,Gasoline,158.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,M/T,Red,Black,0,Yes,6000.0,Manual,2.0,158.0,4.0,5


In [40]:
cols_drop = ['engine','transmission', 'model' ]
df.drop(columns=cols_drop,axis=1, inplace=True)

In [41]:
clean_title_map = {'Yes': 1, 'No': 0}
df['clean_title'] = df['clean_title'].apply(lambda x : clean_title_map[x])

In [42]:
df.sample(5)

,brand,model_year,milage,fuel_type,ext_col,int_col,accident,clean_title,price,transmission_type,engine_litre,hp,cylinders,transmission_speed
3877,BMW,2005,190395.0,Gasoline,Black,Brown,0,1,5000.0,Automatic,2.5,184.0,4.0,5
1999,Lexus,2010,113500.0,Gasoline,Red,Beige,1,1,11000.0,Automatic,3.5,272.0,6.0,5
2115,Dodge,2016,37700.0,Gasoline,Black,Black,1,1,38000.0,Automatic,2.0,300.0,4.0,8
359,Audi,2015,51000.0,Gasoline,White,Black,1,1,102000.0,Manual,5.2,550.0,8.0,5
1150,Ford,2016,107000.0,E85 Flex Fuel,White,Gray,1,1,22000.0,Automatic,5.0,385.0,8.0,6


# Encoding

In [43]:
cols_encode = ['brand', 'fuel_type', 'transmission_type', 'ext_col', 'int_col', 'model_year']
cols_scale = ['milage' , 'hp', 'cylinders', 'transmission_speed', 'engine_litre', 'price']

In [44]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df[cols_encode] = df[cols_encode].apply(le.fit_transform)

In [45]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
df[cols_scale] = sc.fit_transform(df[cols_scale])

In [46]:
df.sample(5)

,brand,model_year,milage,fuel_type,ext_col,int_col,accident,clean_title,price,transmission_type,engine_litre,hp,cylinders,transmission_speed
3052,22,25,-0.453577,2,304,14,0,1,-0.248450,0,0.004701,-0.370174,-0.082046,-0.600719
3629,11,30,-0.473466,2,304,71,1,1,-0.203977,0,1.473101,0.448083,1.190388,1.211382
2052,8,20,0.028350,2,29,14,0,1,-0.356466,0,-0.414842,-0.506550,-0.082046,0.003315
3466,52,31,-0.808194,2,183,14,0,0,-0.057397,0,-1.114081,-0.233798,-0.082046,-0.600719
3282,11,26,-0.028066,2,234,14,0,1,-0.329135,0,0.004701,-0.306532,-0.082046,-0.600719


# Model Train

In [47]:
x = df.drop(columns=['price'])
y = df['price']

In [48]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [49]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(x_train, y_train)

RandomForestRegressor(random_state=42)

In [50]:
from sklearn.tree import DecisionTreeRegressor
dtr = DecisionTreeRegressor()
dtr.fit(x_train, y_train)

from sklearn.metrics import mean_absolute_error, mean_squared_error
y_pred = dtr.predict(x_test)
print('MAE:', mean_absolute_error(y_test, y_pred))
print('MSE:', mean_squared_error(y_test, y_pred))
print('RMSE:', np.sqrt(mean_squared_error(y_test, y_pred)))

MAE: 0.3029342656000765
MSE: 3.1213876978162207
RMSE: 1.7667449441886682


In [51]:
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor()
knn.fit(x_train, y_train)

from sklearn.metrics import mean_absolute_error, mean_squared_error
y_pred = knn.predict(x_test)
print('MAE:', mean_absolute_error(y_test, y_pred))
print('MSE:', mean_squared_error(y_test, y_pred))
print('RMSE:', np.sqrt(mean_squared_error(y_test, y_pred)))

MAE: 0.33836811978969544
MSE: 3.2179530935018748
RMSE: 1.7938654056260395


In [52]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
y_pred = model.predict(x_test)
print('MAE:', mean_absolute_error(y_test, y_pred))
print('MSE:', mean_squared_error(y_test, y_pred))
print('RMSE:', np.sqrt(mean_squared_error(y_test, y_pred)))

MAE: 0.24171987525241484
MSE: 2.9181597057994306
RMSE: 1.7082621888338543


In [54]:
x_train.sample(5)

,brand,model_year,milage,fuel_type,ext_col,int_col,accident,clean_title,transmission_type,engine_litre,hp,cylinders,transmission_speed
1162,44,28,-0.631794,2,304,14,1,1,0,0.004701,-0.415632,-0.082046,-0.600719
3269,50,21,0.789489,2,29,14,0,1,1,-0.764462,-0.552009,-1.354481,-0.600719
741,11,19,0.726016,2,234,14,0,1,1,-1.114081,-1.524825,-1.354481,-0.600719
3089,15,19,0.320949,2,48,71,0,1,0,-0.834385,-1.306623,-1.354481,-0.600719
180,43,27,-1.151051,2,29,14,0,1,0,0.144548,-0.233798,-0.082046,0.607348


In [ ]:
import joblib
joblib.dump(model, 'car_price_model.pkl')